In [1]:
from google.colab import drive
#drive.mount("/content/gdrive", force_remount=True)
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools

# Set paths to input and output data
#INPUT_DIR = '/content/gdrive/MyDrive/data/vox1_test_wav/wav/id10270/5r0dWxy17C8'
OUTPUT_DIR = '/content/gdrive/MyDrive/t3'

# Print names of 10 WAV files from the input path
# parent_list = os.listdir(INPUT_DIR)
# for i in range(10):
#     print(parent_list[i])

In [ ]:
# Declare constants
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 64
N_CHANNELS = 3
N_CLASSES = 40

# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="validation",
                                             seed=0)

Found 4883 files belonging to 40 classes.
Using 3907 files for training.
Found 4883 files belonging to 40 classes.
Using 976 files for validation.


In [ ]:
# Function to prepare our datasets for modelling
def prepare(ds, augment=False):
    # Define our one transformation
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])
    
    #Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: 
      ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False)
valid_dataset = prepare(valid_dataset, augment=False)

In [ ]:

# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))


# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

checkpoint_filepath = '/content/gdrive/MyDrive/tmp/checkpoint/trying_archi/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# Train model for 10 epochs, capture the history
history = model.fit(train_dataset, epochs=10, validation_data=valid_dataset, callbacks=[model_checkpoint_callback])

In [ ]:

# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.005))
#model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.05))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# checkpoint_path='/content/gdrive/MyDrive/tmp/checkpoint/trying_archi/weights.08-2.08.hdf5'
# model.load_weights(checkpoint_path)


# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)


checkpoint_filepath = '/content/gdrive/MyDrive/tmp/checkpoint/trying_archi/from start/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# Train model for 10 epochs, capture the history
history = model.fit(train_dataset, epochs=100, validation_data=valid_dataset, callbacks=[model_checkpoint_callback])

In [ ]:

# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.005))
#model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.05))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

checkpoint_path='/content/gdrive/MyDrive/tmp/checkpoint/op weights a/weights.42-1.05.hdf5'
model.load_weights(checkpoint_path)


# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam',
    ),
    metrics=['accuracy'],
)


checkpoint_filepath = '/content/gdrive/MyDrive/tmp/checkpoint/trying_archi/after 42 adam/second run/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# Train model for 10 epochs, capture the history
history = model.fit(train_dataset, epochs=8, validation_data=valid_dataset, callbacks=[model_checkpoint_callback])

In [ ]:

# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.005))
#model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.05))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

checkpoint_path='/content/gdrive/MyDrive/tmp/checkpoint/op weights a/weights.42-1.05.hdf5'
model.load_weights(checkpoint_path)


# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam',
    ),
    metrics=['accuracy'],
)


checkpoint_filepath = '/content/gdrive/MyDrive/tmp/checkpoint/trying_archi/after 42 adam/second run/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# Train model for 10 epochs, capture the history
history = model.fit(train_dataset, epochs=1, validation_data=valid_dataset, callbacks=[model_checkpoint_callback])

62/62 [==============================] - 1229s 20s/step - loss: 0.7229 - accuracy: 0.7610 - val_loss: 1.0143 - val_accuracy: 0.7346


In [ ]:
model.evaluate(train_dataset, verbose=1)

62/62 [==============================] - 22s 311ms/step - loss: 0.1582 - accuracy: 0.9759


[0.15817782282829285, 0.9759406447410583]

In [ ]:
model.evaluate(valid_dataset, verbose=1)

16/16 [==============================] - 5s 174ms/step - loss: 1.0143 - accuracy: 0.7346


[1.014265775680542, 0.7346311211585999]

In [ ]:
final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Final loss: 1.014266, final accuracy: 0.734631


In [ ]:
model.predict(train_dataset, verbose=1)

62/62 [==============================] - 22s 306ms/step


array([[3.58411412e-10, 9.03755063e-05, 8.42600421e-04, ...,
        2.07468224e-06, 1.18364629e-09, 1.76585206e-06],
       [1.77458950e-20, 8.60816414e-08, 8.65279029e-12, ...,
        5.80467510e-17, 2.80201267e-26, 1.07496634e-16],
       [1.12748451e-08, 9.06327280e-09, 7.66616270e-10, ...,
        4.19729460e-08, 2.77601431e-09, 3.02615621e-09],
       ...,
       [3.56871234e-11, 1.42721092e-05, 5.95565064e-09, ...,
        6.35487702e-13, 2.61182338e-14, 9.43228588e-05],
       [2.02083399e-13, 1.57924787e-05, 3.40364932e-05, ...,
        3.60477065e-11, 3.87222138e-17, 5.33863204e-04],
       [1.04245612e-24, 1.10753418e-09, 9.06865506e-20, ...,
        3.24195578e-25, 1.47011198e-29, 6.54804029e-15]], dtype=float32)

In [ ]:
history = model.fit(train_dataset, epochs=5, validation_data=valid_dataset, callbacks=[model_checkpoint_callback])

Epoch 1/5
62/62 [==============================] - 30s 444ms/step - loss: 0.6831 - accuracy: 0.7840 - val_loss: 0.9963 - val_accuracy: 0.7418
Epoch 2/5
62/62 [==============================] - 30s 445ms/step - loss: 0.6759 - accuracy: 0.7801 - val_loss: 0.9823 - val_accuracy: 0.7428
Epoch 3/5
62/62 [==============================] - 30s 449ms/step - loss: 0.6626 - accuracy: 0.7919 - val_loss: 0.9782 - val_accuracy: 0.7469
Epoch 4/5
62/62 [==============================] - 31s 462ms/step - loss: 0.6835 - accuracy: 0.7789 - val_loss: 0.9694 - val_accuracy: 0.7490
Epoch 5/5
62/62 [==============================] - 31s 453ms/step - loss: 0.6827 - accuracy: 0.7814 - val_loss: 0.9689 - val_accuracy: 0.7510
